# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import ipywidgets

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
weather_df = pd.read_csv('../WeatherPy/weather_data.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
# Latitude and longitude as locations.

latitude_longitude = weather_df[['lat', 'lng']]

# Humidity as weight.

humidity = weather_df["humidity"]

latitude_longitude.head()

,lat,lng
0,29.98,76.82
1,47.80,67.71
2,47.80,67.71
3,46.32,129.57
4,63.75,-68.51


In [32]:
# Add Heatmap layer to map.
figure_layout = {
    'width': '500px',
    'height': '400px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure

fig = gmaps.figure(layout=figure_layout,zoom_level=3,center=(25,25))

# Create heat layer

heat_layer = gmaps.heatmap_layer(latitude_longitude, weights=humidity, 
                                 
                                 dissipating=False, max_intensity=100,
                                 
                                 point_radius=1.5)

# Add heat layer

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='400px', margin='0 auto 0 auto', padding='1px', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
# Create the perfect vacation climate

# A max temperature lower than 80 degrees but higher than 70.

perfect_temperature = (weather_df.temperature < 280) & (weather_df.temperature > 240)

perfect_wind = weather_df.wind_speed < 20

perfect_cloudiness = weather_df.cloudiness <= 50

perfect_vacation = perfect_temperature & perfect_wind & perfect_cloudiness 

# Use boolean indexing to filter the weather_df dataframe - drop null values

ideal_weather = weather_df[perfect_vacation]

ideal_weather = ideal_weather.dropna()

ideal_weather = ideal_weather.reset_index()

ideal_weather.head(10)

,index,date,city_number,city_name,country,lat,lng,temperature,cloudiness,wind_speed,humidity
0,9,01/03/21,1218420,Seydi,TM,39.48,62.91,271.84,3.23,0,77
1,10,01/03/21,1218420,Seydi,TM,39.48,62.91,271.84,3.23,0,77
2,16,01/03/21,2017364,Russkiy,RU,43.01,131.86,249.15,6.99,0,69
3,28,01/03/21,604490,Luleå,SE,65.58,22.15,261.15,1.50,0,92
4,41,01/03/21,1490551,Suzun,RU,53.78,82.31,243.61,1.05,0,85


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = ideal_weather

# Add column for Hotel Name

hotel_df['Hotel Name'] = ""

hotel_df.head()

,index,date,city_number,city_name,country,lat,lng,temperature,cloudiness,wind_speed,humidity,Hotel Name
0,9,01/03/21,1218420,Seydi,TM,39.48,62.91,271.84,3.23,0,77,
1,10,01/03/21,1218420,Seydi,TM,39.48,62.91,271.84,3.23,0,77,
2,16,01/03/21,2017364,Russkiy,RU,43.01,131.86,249.15,6.99,0,69,
3,28,01/03/21,604490,Luleå,SE,65.58,22.15,261.15,1.50,0,92,
4,41,01/03/21,1490551,Suzun,RU,53.78,82.31,243.61,1.05,0,85,


In [35]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)


    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.info()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        5 non-null      int64  
 1   date         5 non-null      object 
 2   city_number  5 non-null      int64  
 3   city_name    5 non-null      object 
 4   country      5 non-null      object 
 5   lat          5 non-null      float64
 6   lng          5 non-null      float64
 7   temperature  5 non-null      float64
 8   cloudiness   5 non-null      float64
 9   wind_speed   5 non-null      int64  
 10  humidity     5 non-null      int64  
 11  Hotel Name   5 non-null      object 
dtypes: float64(4), int64(4), object(4)
memory usage: 608.0+ bytes


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [37]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…